<a href="https://colab.research.google.com/github/Dalia-Sher/Generating-Facial-Expressions-Wasserstein-Conditional-BiGAN/blob/Shir/biGAN_with_combined_disc_and_gen_10000_epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K
import tensorflow as tf

import matplotlib.pyplot as plt
import pandas as pd

import numpy as np

In [14]:
data = pd.read_csv('fer2013.csv')
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [15]:
data.Usage.value_counts()

Training       28709
PrivateTest     3589
PublicTest      3589
Name: Usage, dtype: int64

In [16]:
num_classes = 7
img_width = 48
img_height = 48

In [17]:
data.shape

(35887, 3)

In [18]:
X = data['pixels']
y = data['emotion']

In [19]:
X_train = []
for i in X:
    X_train.append([int(j) for j in i.split()])

X_train = np.array(X_train)/255.0

In [20]:
X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 1)
X_train = X_train.astype('float32')

In [21]:
X_train.shape

(35887, 48, 48, 1)

In [22]:
img_rows, img_cols, channels, latent_dim = 48, 48 , 1 , 100

In [23]:
img_shape = (img_rows, img_cols, channels)


In [24]:
def build_encoder(latent_dim, img_shape):
        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(latent_dim))

        model.summary()

        img = Input(shape=img_shape)
        z = model(img)

        return Model(img, z)

In [68]:
def define_generator(latent_dim, img_shape):
    model = Sequential()
    # foundation for 12x12 image
    n_nodes = 128 * 12 * 12
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((12, 12, 128)))
    # upsample to 24x24
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 48x48
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # generate
    model.add(Conv2D(1, (12, 12), activation='tanh', padding='same'))
    model.add(Reshape(img_shape))

    #model.summary()
    z = Input(shape=(latent_dim,))
    gen_img = model(z)
    return Model(z, gen_img)

In [26]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [27]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# select images
	X = dataset[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return X, y

In [71]:
# define the standalone discriminator model
def define_discriminator(latent_dim, img_shape):

    z = Input(shape=(latent_dim, ))
    img = Input(shape=img_shape)
    d_in = concatenate([z, Flatten()(img)])
    d_shape = d_in.shape

    model = Sequential()
    model.add(tf.keras.layers.Reshape((-1,48,48,1), input_shape=d_shape))
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same', input_shape=d_shape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    
    # classifier
    #model.add(d_in)
    #model.add(Flatten())
    model.add(Dropout(0.4))

    model = Dense(1024)(d_in)
    model = LeakyReLU(alpha=0.2)(model)
    model = Dropout(0.5)(model)
    model = Dense(1024)(model)
    model = LeakyReLU(alpha=0.2)(model)
    model = Dropout(0.5)(model)
    model = Dense(1024)(model)
    model = LeakyReLU(alpha=0.2)(model)
    model = Dropout(0.5)(model)

    #model.add(Dense(1, activation='sigmoid'))
    #validity
    validity = Dense(1, activation="sigmoid")(model)
    return Model([z, img], validity)

    # # compile model
    # opt = Adam(lr=0.0002, beta_1=0.5)
    # model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    # return model

In [73]:
opt = Adam(lr=0.0002, beta_1=0.5)
# Build and compile the discriminator
discriminator = define_discriminator(latent_dim,img_shape)
discriminator.compile(loss=['binary_crossentropy'],
    optimizer=opt,
    metrics=['accuracy'])

# Build the generator
generator = define_generator(latent_dim, img_shape)

# Build the encoder
encoder = build_encoder(latent_dim, img_shape)

# The part of the bigan that trains the discriminator and encoder
discriminator.trainable = False

# Generate image from sampled noise
z = Input(shape=(latent_dim, ))
img_ = generator(z)

# Encode image
img = Input(shape=img_shape)
z_ = encoder(img)

# Latent -> img is fake, and img -> latent is valid
fake = discriminator([z, img_])
valid = discriminator([z_, img])

# Set up and compile the combined model
# Trains generator to fool the discriminator
bigan_generator = Model([z, img], [fake, valid])
bigan_generator.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
    optimizer=opt)

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_17 (Flatten)         (None, 2304)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 512)               1180160   
_________________________________________________________________
leaky_re_lu_32 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 512)               2048      
_________________________________________________________________
dense_23 (Dense)             (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_33 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 512)             

In [74]:
def train( X_train, bigan_generator, generator, discriminator, encoder,latent_dim, epochs, batch_size=128, intervals=100):

  # Adversarial ground truths
  valid = np.ones((batch_size, 1))
  fake = np.zeros((batch_size, 1))

  for epoch in range(epochs):
    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Sample noise and generate img
    z = np.random.normal(size=(batch_size, latent_dim))
    imgs_ = generator.predict(z)

    # Select a random batch of images and encode
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]
    z_ = encoder.predict(imgs)

    # Train the discriminator (img -> z is valid, z -> img is fake)
    d_loss_real = discriminator.train_on_batch([z_, imgs], valid)
    d_loss_fake = discriminator.train_on_batch([z, imgs_], fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    #  Train Generator
    # ---------------------

    # Train the generator (z -> img is valid and img -> z is is invalid)
    g_loss = bigan_generator.train_on_batch([z, imgs], [valid, fake])

    # Plot the progress
    print ("%d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0]))

    # If at save interval => save generated image samples
    if epoch % intervals == 0:
        sample_interval(latent_dim, epoch)

In [75]:
def sample_interval(latent_dim, epoch):
  r, c = 5, 5
  z = np.random.normal(size=(25, latent_dim))
  gen_imgs = generator.predict(z)

  gen_imgs = 0.5 * gen_imgs + 0.5

  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
      for j in range(c):
          axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
          axs[i,j].axis('off')
          cnt += 1
  fig.savefig(r"C:\Users\shir2\Desktop\Shir\MSc\Deep_generative_models\project\biGan100321\expression_facial_new%d.png" % epoch)
  plt.close()

In [76]:
train(X_train, bigan_generator, generator, discriminator, encoder,latent_dim, epochs=10000, batch_size=32, intervals=500)

Streaming output truncated to the last 5000 lines.
4997 [D loss: 0.698984, acc: 48.44%] [G loss: 1.370322]
4998 [D loss: 0.714520, acc: 37.50%] [G loss: 1.418291]
4999 [D loss: 0.691373, acc: 57.81%] [G loss: 1.392612]
5000 [D loss: 0.694590, acc: 50.00%] [G loss: 1.382243]
5001 [D loss: 0.702683, acc: 46.88%] [G loss: 1.403057]
5002 [D loss: 0.709464, acc: 34.38%] [G loss: 1.404373]
5003 [D loss: 0.711393, acc: 39.06%] [G loss: 1.394479]
5004 [D loss: 0.703322, acc: 42.19%] [G loss: 1.404379]
5005 [D loss: 0.703083, acc: 40.62%] [G loss: 1.386133]
5006 [D loss: 0.714562, acc: 43.75%] [G loss: 1.392115]
5007 [D loss: 0.704680, acc: 43.75%] [G loss: 1.385810]
5008 [D loss: 0.695667, acc: 51.56%] [G loss: 1.402080]
5009 [D loss: 0.698865, acc: 39.06%] [G loss: 1.402461]
5010 [D loss: 0.687881, acc: 48.44%] [G loss: 1.396397]
5011 [D loss: 0.686448, acc: 54.69%] [G loss: 1.361932]
5012 [D loss: 0.696218, acc: 43.75%] [G loss: 1.414682]
5013 [D loss: 0.705594, acc: 43.75%] [G loss: 1.40538